<a href="https://colab.research.google.com/github/Alexidis/gb_torch/blob/lesson_9/hw9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
!pip install transformers stop_words pymorphy2 tqdm nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [92]:
import numpy as np
import pandas as pd
from pathlib import Path

import torch
import torch.nn as nn
# import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm

# from transformers import pipeline
# from transformers import BertTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import BertForSequenceClassification
from transformers import BertTokenizerFast

from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from string import punctuation
import nltk

In [93]:
nltk.download("punkt")
stop_words = set(get_stop_words("ru"))
punctuations = set(punctuation)
lemmatizer = MorphAnalyzer()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'cuda'

In [94]:
tok = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
b_model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

In [95]:
class TwitterDataset(torch.utils.data.Dataset):

    def __init__(self, txts, labels, tokenizer):
        self._labels = labels

        # self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.tokenizer = tokenizer
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]

    def __len__(self):
        return len(self._txts)

    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [96]:
class BertClassifier(nn.Module):

    def __init__(self, bert_model, dropout=0.5):
        super().__init__()
        # self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.Xrobert = bert_model
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(3, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):

        pooled_output = self.Xrobert(input_ids=x, attention_mask=mask, return_dict=False)[0]
        # _, pooled_output - набор эмбеддинигов слов, эмбеддинг предложения
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)

        return final_layer

In [97]:
def preprocess_text(txt, sw, lem, punct):

    txt = str(txt)
    txt = "".join(c for c in txt if c not in punct)
    txt = txt.lower()
    txt = [lem.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [98]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip
!unzip -f data.zip

--2022-09-08 08:36:09--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Resolving drive.google.com (drive.google.com)... 108.177.12.101, 108.177.12.138, 108.177.12.100, ...
Connecting to drive.google.com (drive.google.com)|108.177.12.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hh5ac1gkl42isorn0tm20u5ttdnetntf/1662626100000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=392c6fdc-ee9e-410f-8b0f-c8cf5ecec6c1 [following]
--2022-09-08 08:36:12--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hh5ac1gkl42isorn0tm20u5ttdnetntf/1662626100000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=392c6fdc-ee9e-410f-8b0f-c8cf5ecec6c1
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 142.251.162.132, 2607

In [99]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

df_train.shape, df_val.shape

((181467, 3), (22683, 3))

In [100]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [101]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

In [102]:
tqdm.pandas()

df_train['text'] = df_train['text'].progress_apply(lambda t: preprocess_text(t, stop_words, lemmatizer, punctuations))
df_val['text'] = df_val['text'].progress_apply(lambda t: preprocess_text(t, stop_words, lemmatizer, punctuations))

100%|██████████| 22683/22683 [00:28<00:00, 804.91it/s]


In [103]:
%%time
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train, tok)
valid_dataset = TwitterDataset(df_val['text'], y_val, tok)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

CPU times: user 48.9 s, sys: 14 s, total: 1min 2s
Wall time: 59.1 s


In [104]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])


In [105]:
model = BertClassifier(b_model).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.Xrobert.classifier.parameters(), lr=0.0016)  # неполное обучение

In [106]:
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.Xrobert.classifier.parameters()]))

BertClassifier(
  (Xrobert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias

In [107]:
for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0
    model.train()

    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()

    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()

        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc

    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

100%|██████████| 2836/2836 [05:13<00:00,  9.04it/s]


Epochs: 1 | Train Loss:  0.011         | Train Accuracy:  0.558         | Val Loss:  0.011         | Val Accuracy:  0.582


100%|██████████| 2836/2836 [05:20<00:00,  8.86it/s]


Epochs: 2 | Train Loss:  0.011         | Train Accuracy:  0.559         | Val Loss:  0.011         | Val Accuracy:  0.535
